In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe, table.simpletable{font-size:19px;}
</style>
"""))

- 데이터 소스 : 국토교통부 실거래가 공개시스템
https://rtmobile.molit.go.kr/pt/xls/xls.do?mobileAt=

In [2]:
import pandas as pd          # csv 파일 read
import statsmodels.api as sm # 회귀분석 모델 생성
import joblib     # pkl 모델 저장하거나 load

In [5]:
df = pd.read_csv('../data/trade_apt_api.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   거래금액     318 non-null    int64  
 1   건축년도     318 non-null    int64  
 2   년        318 non-null    int64  
 3   법정동      318 non-null    object 
 4   아파트      318 non-null    object 
 5   월        318 non-null    int64  
 6   일        318 non-null    int64  
 7   전용면적     318 non-null    float64
 8   지번       318 non-null    object 
 9   지역코드     318 non-null    int64  
 10  층        318 non-null    int64  
 11  해제사유발생일  318 non-null    object 
 12  해제여부     318 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 32.4+ KB


In [6]:
df.head(1)

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,80000,2002,2021,신교동,신현(101동),8,16,84.82,6-13,11110,1,-,-


In [12]:
x = df[['건축년도','전용면적','층']]
X = sm.add_constant(x) # 상수항(1) 추가하기 위한 add_constant함수
y = df['거래금액']
print('독립변수 X와 타겟변수 y의 shape :', X.shape, y.shape)

독립변수 X와 타겟변수 y의 shape : (318, 4) (318,)


In [13]:
model = sm.OLS(y, X).fit()
model.summary()
# R-squared : r제곱(설명도) : 독립변수가 종속변수를 설명해주는 수치
# Adj. R-squared:	0.644 : 수정된 r제곱
# Prob (F-statistic): 8.54e-71 f통계량 Prob은 0.05보다 작으면 유의미
# Durbin-Watson:	1.352 : 2에 가까울 수록 상관없음

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   거래금액   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     192.4
Date:                Mon, 06 Jan 2025   Prob (F-statistic):           8.54e-71
Time:                        10:20:25   Log-Likelihood:                -3777.5
No. Observations:                 318   AIC:                             7563.
Df Residuals:                     314   BIC:                             7578.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.855e+06   4.25e+05     -9.069      0.000   -4.69e+06   -3.02e+06
건축년도        1925.6916    212.616      9.057      0.000    1507.360    2344.023
전용면적         962.1507     47.367     20.313      0.000     868.955    1055.347
층           2058.1524    417.716      4.927      0.000    1236.276    2880.028
==============================================================================
Omnibus:                       20.985   Durbin-Watson:                   1.352
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.734
Skew:                           0.345   Prob(JB):                     5.25e-10
Kurtosis:                       4.658   Cond. No.                     4.33e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
X.iloc[0], y[0]

(const       1.00
 건축년도     2002.00
 전용면적       84.82
 층           1.00
 Name: 0, dtype: float64,
 80000)

In [18]:
model.predict([[1, 2002, 84.82, 1]])

array([83789.14484516])

In [19]:
# 모델 저장
joblib.dump(model, '../model/ex1_apt_price_regression.pkl')

['../model/ex1_apt_price_regression.pkl']

In [21]:
format(1212, ',')

'1,212'

In [25]:
def predict_apt_price(year, square, floor):
    loaded_model = joblib.load('../model/ex1_apt_price_regression.pkl')
    input_data = [[1, year, square, floor]]
    results = loaded_model.predict(input_data)
    result = round(results[0]*10000)
    return format(result, ',') + '원입니다'

In [26]:
year   = int(input('몇년 건축?'))
square = int(input('몇 제곱미터?'))
floor  = int(input('몇 층?'))
predict_apt_price(year, square, floor)

몇년 건축?2024
몇 제곱미터?105
몇 층?2


'1,476,287,136원입니다'